# full exp:
- gridsearch blocked/interleaved, using MSE human as eval metric
    - fix parameters
- eval fit on early/middle/late conditions

# handheld splitting, RNN schema

In [1]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from scipy.stats import zscore
from utils import *
import pandas as pd
from glob import glob as glob

In [2]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### human data for plotting

In [3]:
##  load human data for fitting
humandf = pd.read_csv('humandf.csv')
human_accD = {}
condL =['blocked','interleaved','early','middle','late']
for cond in condL:
    human_accD[cond] = humandf.loc[:,'%s mean'%cond].values
# 1d arrs of acc
human_accD['blocked'].shape,human_accD.keys()

((200,), dict_keys(['blocked', 'interleaved', 'early', 'middle', 'late']))

# full experiment

In [4]:
def run_exp(nseeds,condL,paramD,ntr=160,nte=40):
    """ returns acc for cond in condL 
    """
    print('N=%i'%nseeds,paramD)
    acc = -np.ones([len(condL),nseeds,ntr+nte])
    for ci,cond in enumerate(condL):
        for s in range(nseeds):
            # seed ctrl
            np.random.seed(s)
            tr.manual_seed(s)
            # init
            ag = Agent(**paramD)
            task = Task()
            # run
            exp,cur = task.generate_experiment(cond,ntr,nte)
            acc[ci,s] = ag.forward_exp(exp) 
    return acc
  
def plt_human(ax=None,condL=['blocked'],c='k'):
  for cond in condL:
    ax.plot(human_accD[cond],
            c=c,zorder=-99,alpha=0.4
           )
  return None

def plt_acc(acc,condL,ax=None,h=1):
  # model
  for ac in acc:
    ax.plot(ac.mean(0))
  # human plot
  if h:
    plt_human(ax=ax,condL=condL)
  ax.set_ylim(0.2,1)    
  return None

### blocked / interleaved plots

In [5]:
bestparamD = {
  'sticky_decay': 0.02, 
  'pe_thresh': 0.9, 
  'init_lr': 0.45, 
  'lr_decay': 0.25, 
  'stsize': 6.0
}

In [ ]:
ns = 50
condBI = ['blocked','interleaved']
acc_bestBI = run_exp(nseeds=ns,
              condL=condBI,
              paramD=bestparamD
             )

N=50 {'sticky_decay': 0.02, 'pe_thresh': 0.9, 'init_lr': 0.45, 'lr_decay': 0.25, 'stsize': 6.0}


In [ ]:
plt.subplots(figsize=(10,4))
ax = plt.gca()
plt_acc(acc_bestBI,condBI,ax=ax)

### early / middle / late plots

In [ ]:
condEML=['early','middle','late']
ns = 50
acc_bestEML = run_exp(nseeds=ns,
              condL=condEML,
              paramD=bestparamD
             )

In [ ]:
f,ax = plt.subplots(3,1,figsize=(10,10),sharex=True)
for ci in range(3):
  cond = condEML[ci]
  plt_acc([acc_bestEML[ci]],[cond],ax=ax[ci])
  ax[ci].set_title(cond)

# analysis of splitting